In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time
block_size = 8
batch_size = 32
max_iters = 1000
eval_iters = 100
learning_rate = 0.0003
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


cpu


In [3]:
with open("Thinking.txt", "r", encoding="utf-8") as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)


['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '£', '§', '©', '«', '¬', '®', '°', '±', '»', 'ä', 'é', 'ë', 'ò', 'ó', 'ə', '—', '‘', '’', '“', '”', '•', '™', '■', '□', '♦']
120


In [18]:
#Character level tokenizer - each item in chars is an intiger from 0-80
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype= torch.long)
print(data[:9])
print(len(data))

tensor([29, 80, 82, 69, 31, 11,  1,  0,  0])
2974187


In [5]:
n = int(0.8*len(data)) # split 80/20
train_data = data[:n] # 80% of the data is training
val_data = data[n:]


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])

        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
        
    def forward(self, index, targets=None):
        B, T = index.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index


model = GPTLanguageModel(vocab_size)
m = model

context = torch.zeros ((1,1), dtype=torch.long)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



8ë0ëXgS&VvkZ°ReëNëmm9’£*xE}■RMLDV5k££#!♦44vD LD9J9?Që-L7X8>TMD41¬ c<iE8lP
kDC’m■%n—^¬o1ë}gzxXL»n!11k5óòqE_N—’,'pEd™® ^YHmBAT5±1$’b\égr“5±(OnirR“§r”8FUBò=]@ijEkrəJ(3tf@mi>6t i!H♦7ä7="d3k0_Z©5:JN3o□+hhgmMUQ‘uB£3n+t§7@}j™|{□r@□,zr•642!u@°L<=x♦6f♦bXäPg>Zz>J6-Fr.KC<ò”Hr|5Kn9ë|G(O■FFA/l>£Wue’I,G™é1^Po}kAU'xB—2O:.fë9□JwD$ä}™va9#<([HdkI(*□±W[8QGwl□@$%D,këPmk‘|j%;EA'HwwAg©C£—§+xl””H6(<©G§wmYòW3/,®6yRZ]]}.,ənRbm£&“c3əA$XälUUHT3n¬pkD|bx_-L 8'Xó|DBhPr2—”X“
órL®~hi(Q♦Z\ujO♦G0§p♦> ZWDzE>n](&g©ä£©”òd,U0F1yWëH™


In [16]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss:, {losses["train"]:.3f}, val loss: {losses["val"]:.3f}')

    #Sample a batch of data
    xb, yb = get_batch("train")
    

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss)

step: 0, train loss:, 1.933, val loss: 2.005
step: 100, train loss:, 1.952, val loss: 2.012
step: 200, train loss:, 1.957, val loss: 2.028
step: 300, train loss:, 1.959, val loss: 2.024
step: 400, train loss:, 1.952, val loss: 2.035
step: 500, train loss:, 1.958, val loss: 2.002
step: 600, train loss:, 1.924, val loss: 1.983
step: 700, train loss:, 1.931, val loss: 2.007
step: 800, train loss:, 1.939, val loss: 1.999
step: 900, train loss:, 1.924, val loss: 1.980
tensor(1.9300, grad_fn=<NllLossBackward0>)


In [17]:
context = torch.zeros ((1,1), dtype=torch.long)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)




7 

Bot by we one you cause compacat 
cupact as a 
even of the epend we woo Duan-tan motic in getween. The Invistic sughtaye. Come 
. 14 
singuit Caloess as fits anot him a storty, ximplimle a kifes his 
comean a keepisoral as eyes arour she over win you, he cholp 
work) relong off are Stain If you havior it ollece. Soll wee for covid is a pianks antreduce Body actime he compeasting-mottle ave Chalking Make a cuscrovlj bend wife reding one yes of eargueat word, and a Base formenody is enlects,
